In [1]:
%load_ext kedro.extras.extensions.ipython

[02/12/24 18:16:27] WARNING  /home/smartroom/miniconda3/envs/raftel/lib/python3.8/site-packages/ked ]8;id=62546;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=965965;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/warnings.py#109\109]8;;\
                             ro/extras/extensions/ipython.py:18: DeprecationWarning:                               
                             kedro.extras.extensions.ipython should be accessed only using the                     
                             alias kedro.ipython. The unaliased name will be removed in Kedro                      
                             0.19.0.                                                                               
                               warnings.warn(                                                                      
                                                                                                                   

                    INFO     Resolved project path as: /home/smartroom/jhonatan/raftel.             ]8;id=315700;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=573649;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#134\134]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

your 131072x1 screen size is bogus. expect trouble
24/02/12 18:16:31 WARN Utils: Your hostname, WIN-0B8Q3RIVNP0 resolves to a loopback address: 127.0.1.1; using 172.31.114.225 instead (on interface eth0)
24/02/12 18:16:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/smartroom/miniconda3/envs/raftel/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/smartroom/.ivy2/cache
The jars for the packages stored in: /home/smartroom/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
software.aws.rds#aws-mysql-jdbc added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7c388cb7-917a-4165-b740-441ada773ca5;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.hadoop#hadoop-aws;3.3.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.1026 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found software.aws.rds#aws-mysql-jdbc;1.0.0 in central
:: resolution report :: resolve 363ms :: artifacts dl 25ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.1026 from central in [default]
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta

http://172.31.114.225:4054


[02/12/24 18:16:39] INFO     Kedro project raftel                                                   ]8;id=83471;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=816886;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#103\103]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=503535;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=539836;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#104\104]8;;\
                             'pipelines'                                                                           

[02/12/24 18:16:42] WARNING  /home/smartroom/miniconda3/envs/raftel/lib/python3.8/site-packages/ked ]8;id=235477;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=630130;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/warnings.py#109\109]8;;\
                             ro_viz/api/graphql/router.py:10: DeprecationWarning: The `graphiql`                   
                             argument is deprecated in favor of `graphql_ide`                                      
                               graphql_app: GraphQL = GraphQL(schema, graphiql=False)                              
                                                                                                                   

[02/12/24 18:16:43] INFO     Registered line magic 'run_viz'                                        ]8;id=392182;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=646033;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#110\110]8;;\

In [2]:
%reload_kedro

                    INFO     Resolved project path as: /home/smartroom/jhonatan/raftel.             ]8;id=814714;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=580395;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#134\134]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[02/12/24 18:16:43] INFO     Kedro project raftel                                                   ]8;id=746470;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=417218;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#103\103]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=364082;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=809797;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#104\104]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=291153;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=506546;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#110\110]8;;\

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import pyspark.sql.functions as F
from pyspark.sql.window import Window
import math

In [ ]:
master_table = catalog.load("master_table@spark")

### Create cyclical features for the time

In [ ]:
master_table = master_table.withColumn(
    "SEC2",
    F.col("SEC") - (F.col("DAY") - 1) *  86400
).withColumn(
    "SEC_SIN",
    F.sin(2 * math.pi * F.col("SEC2")/86400)
).withColumn(
    "SEC_COS",
    F.cos(2 * math.pi * F.col("SEC2")/86400)
)

### Create dummies for DAY

In [ ]:
master_table = master_table.withColumn(
    "DAY_OF_WEEK",
    F.when(
        F.col("DAY") % 7 == 0, 7
    ).otherwise(
    F.expr("CAST(DAY AS INT) % 7")
    )
)

In [ ]:
expresiones_columnas_no_numericas = []
columna = "DAY_OF_WEEK"

unicos_columna = (
    master_table.select(columna)
    .distinct()
    .rdd.flatMap(lambda x: x)
    .collect()
)

unicos_columna.sort()

expr_unicos_columna = [
    F.when(
        F.col(columna)
        == valor_unico,
        1,
    )
    .otherwise(0)
    .alias(
        "{columna}_{valor}".format(
            columna=columna,
            valor=int(valor_unico),
        )
    )
    for valor_unico in unicos_columna
    if valor_unico != 1 #tiene un efecto parecido que drop_first=True
]

expresiones_columnas_no_numericas += expr_unicos_columna

In [ ]:
master_table = master_table.select("*", *expresiones_columnas_no_numericas)

In [ ]:
master_table = master_table.select(
    'SEC_SIN',
    'SEC_COS',
    'Ph1',
    'Ph2',
    'Ir1',
    'Fo1',
    'Fo2',
    'Di3',
    'Di4',
    'Ph3',
    'Ph4',
    'Ph5',
    'Ph6',
    'Co1',
    'Co2',
    'Co3',
    'So1',
    'So2',
    'Di1',
    'Di2',
    'Te1',
    'Fo3',
    'DAY_OF_WEEK_2',
    'DAY_OF_WEEK_3',
    'DAY_OF_WEEK_4',
    'DAY_OF_WEEK_5',
    'DAY_OF_WEEK_6',
    'DAY_OF_WEEK_7',
    'Lag3_LABEL',
    'Lag2_LABEL',
    'Lag1_LABEL',
    'LABEL'
)

In [ ]:
catalog.save("featured_master_table@spark", master_table)

In [ ]:
master_table.groupby("LABEL").agg(F.count("LABEL").alias("COUNT")).filter(F.col("COUNT")<10).show()

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, train_test_split, KFold, StratifiedKFold
import xgboost as xgb
from sklearn.metrics import accuracy_score
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.preprocessing import LabelEncoder

[02/12/24 18:16:44] WARNING  /home/smartroom/miniconda3/envs/raftel/lib/python3.8/site-packages/fut ]8;id=469145;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=325932;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/warnings.py#109\109]8;;\
                             ure/standard_library/__init__.py:65: DeprecationWarning: the imp                      
                             module is deprecated in favour of importlib; see the module's                         
                             documentation for alternative uses                                                    
                               import imp                                                                          
                                                                                                                   

### Train and test split

In [5]:
X_aras = catalog.load("featured_master_table@pandas")

                    INFO     Loading data from 'featured_master_table@pandas'                   ]8;id=548297;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=660454;file:///home/smartroom/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/io/data_catalog.py#343\343]8;;\
                             (ParquetDataSet)...                                                                   

In [6]:
y_aras = X_aras['LABEL']
X_aras = X_aras.drop(columns=['LABEL'])

In [7]:
X_aras_train, X_aras_test, y_aras_train, y_aras_test = train_test_split(X_aras, y_aras, test_size = 0.3, random_state = 0, stratify = y_aras)

In [8]:
X_aras_train.shape, X_aras_test.shape, y_aras_train.shape, y_aras_test.shape

((1811203, 31), (776230, 31), (1811203,), (776230,))

In [ ]:
df = y_aras_train.to_frame().groupby(['LABEL']).agg({'LABEL': ['count']})
df[df.LABEL['count'] < 10]

In [ ]:
df = y_aras_test.to_frame().groupby(['LABEL']).agg({'LABEL': ['count']})
df[df.LABEL['count'] < 10]

## Experiments

### 1. Model with XGBoost and encoded activities in one label. No windowing.

#### Cross Validation

In [9]:
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'learning_rate': hp.uniform ('learning_rate', 0,0.5),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0,
    }

In [10]:
def objective(space):
    clf=xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']), eval_metric="auc", early_stopping_rounds=10,
                    learning_rate=space['learning_rate']
    )
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation,
            verbose=False)
    

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [11]:
skf = StratifiedKFold(n_splits=3)
best_hyperparams = []

for i, (train_index, test_index) in enumerate(skf.split(X_aras_train, y_aras_train)):
    print(f"Fold {i}:")

    X_train = X_aras_train.iloc[train_index]
    y_train = y_aras_train.iloc[train_index]
    
    X_test = X_aras_train.iloc[test_index]
    y_test = y_aras_train.iloc[test_index]

    trials = Trials()
    best_hyperparams += [fmin(fn = objective,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 100,
                            trials = trials)]
    print(f"Params {best_hyperparams[-1]}")

Fold 0:
SCORE:                                                 
0.19576966715529165                                    
SCORE:                                                                                
0.8137610044141883                                                                    
SCORE:                                                                                 
0.2749550713475283                                                                     
SCORE:                                                                                 
0.19672372812575054                                                                    
SCORE:                                                                                 
0.001603352464243418                                                                    
SCORE:                                                                                  
0.2641009714527069                                                                      

In [15]:
print("The best hyperparameters are : ","\n")

for i, j in enumerate(best_hyperparams):
    print(f"fold {i+1}: {j}")

The best hyperparameters are :  

fold 1: {'colsample_bytree': 0.8084956082707384, 'gamma': 2.6899288117652205, 'learning_rate': 0.1731791560786223, 'max_depth': 12.0, 'min_child_weight': 5.0, 'reg_alpha': 40.0, 'reg_lambda': 0.05105068156107444}
fold 2: {'colsample_bytree': 0.6573350248910703, 'gamma': 1.584753219875135, 'learning_rate': 0.1643677300411655, 'max_depth': 14.0, 'min_child_weight': 1.0, 'reg_alpha': 46.0, 'reg_lambda': 0.6395744203920048}
fold 3: {'colsample_bytree': 0.9480518173313758, 'gamma': 4.315657553688032, 'learning_rate': 0.2100439767588583, 'max_depth': 17.0, 'min_child_weight': 8.0, 'reg_alpha': 40.0, 'reg_lambda': 0.9800987298988849}


#### Select best hyperparameters for all folds

In [9]:
best_hyperparams = [
    {'colsample_bytree': 0.8084956082707384, 'gamma': 2.6899288117652205, 'learning_rate': 0.1731791560786223, 'max_depth': 12, 'min_child_weight': 5, 'reg_alpha': 40, 'reg_lambda': 0.05105068156107444},
    {'colsample_bytree': 0.6573350248910703, 'gamma': 1.584753219875135, 'learning_rate': 0.1643677300411655, 'max_depth': 14, 'min_child_weight': 1, 'reg_alpha': 46, 'reg_lambda': 0.6395744203920048}, 
    {'colsample_bytree': 0.9480518173313758, 'gamma': 4.315657553688032, 'learning_rate': 0.2100439767588583, 'max_depth': 17, 'min_child_weight': 8, 'reg_alpha': 40, 'reg_lambda': 0.9800987298988849}
    ]

In [10]:
skf = StratifiedKFold(n_splits=3)
skfolds = skf.split(X_aras_train, y_aras_train)

X_trains = []
y_trains = []
X_tests = []
y_tests = []

for i, (train_index, test_index) in enumerate(skfolds):

    X_trains += [X_aras_train.iloc[train_index].values]
    y_trains += [y_aras_train.iloc[train_index].values]
    
    X_tests += [X_aras_train.iloc[test_index].values]
    y_tests += [y_aras_train.iloc[test_index].values]

In [11]:
hypers = best_hyperparams[0]

for i, (X_train, y_train, X_test, y_test) in enumerate(zip(X_trains, y_trains, X_tests, y_tests)):

    clf=xgb.XGBClassifier(
        **hypers, eval_metric="auc", early_stopping_rounds=10, n_estimators=180
    )

    evaluation = [( X_train, y_train), ( X_test, y_test)]

    clf.fit(X_train, y_train,
            eval_set=evaluation,
            verbose=False)

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred)
    print (f"Fold {i} score: {accuracy}")

Fold 0 score: 0.814383794214349
Fold 1 score: 0.8764439306052003
Fold 2 score: 0.8771611338768398


In [12]:
hypers = best_hyperparams[1]

for i, (X_train, y_train, X_test, y_test) in enumerate(zip(X_trains, y_trains, X_tests, y_tests)):

    clf=xgb.XGBClassifier(
        **hypers, eval_metric="auc", early_stopping_rounds=10, n_estimators=180
    )

    evaluation = [( X_train, y_train), ( X_test, y_test)]

    clf.fit(X_train, y_train,
            eval_set=evaluation,
            verbose=False)

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred)
    print (f"Fold {i} score: {accuracy}")

Fold 0 score: 0.939008008480542
Fold 1 score: 0.9388621478995717
Fold 2 score: 0.9407388021877184


In [13]:
hypers = best_hyperparams[2]

for i, (X_train, y_train, X_test, y_test) in enumerate(zip(X_trains, y_trains, X_tests, y_tests)):

    clf=xgb.XGBClassifier(
        **hypers, eval_metric="auc", early_stopping_rounds=10, n_estimators=180
    )

    evaluation = [( X_train, y_train), ( X_test, y_test)]

    clf.fit(X_train, y_train,
            eval_set=evaluation,
            verbose=False)

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred)
    print (f"Fold {i} score: {accuracy}")

Fold 0 score: 0.8709665664571377
Fold 1 score: 0.8701994587020111
Fold 2 score: 0.8707328061696045


In [14]:
for i, (X_train, y_train, X_test, y_test) in enumerate(zip(X_trains, y_trains, X_tests, y_tests)):

    clf=xgb.XGBClassifier(
        eval_metric="auc", early_stopping_rounds=10, n_estimators=180
    )

    evaluation = [( X_train, y_train), ( X_test, y_test)]

    clf.fit(X_train, y_train,
            eval_set=evaluation,
            verbose=False)

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred)
    print (f"Fold {i} score: {accuracy}")

Fold 0 score: 0.8875317813279004
Fold 1 score: 0.8934977986994272
Fold 2 score: 0.8926050214167166


### Not used ↓↓↓↓

In [ ]:
X_aras_train = X_aras.loc[(
    (X_aras['DAY_26'] != 1) & 
    (X_aras['DAY_27'] != 1) & 
    (X_aras['DAY_28'] != 1) & 
    (X_aras['DAY_29'] != 1) &
    (X_aras['DAY_30'] != 1) 
)]

X_aras_test = X_aras.loc[(
    (X_aras['DAY_26'] == 1) | 
    (X_aras['DAY_27'] == 1) | 
    (X_aras['DAY_28'] == 1) | 
    (X_aras['DAY_29'] == 1) | 
    (X_aras['DAY_30'] == 1) 
)]

In [ ]:
y_aras_train = X_aras_train['LABEL']
y_aras_test = X_aras_test['LABEL']


X_aras_train = X_aras_train.drop(columns=['LABEL'])
X_aras_test = X_aras_test.drop(columns=['LABEL'])

In [ ]:
X_aras_train.shape, y_aras_train.shape, X_aras_test.shape, y_aras_test.shape

In [ ]:
from sklearn.datasets import make_multilabel_classification

In [ ]:
X, y = make_multilabel_classification(n_samples=3000, n_features=45, n_classes=20, n_labels=1,
                                      allow_unlabeled=False, random_state=42)

In [ ]:
X